In [24]:
import pandas as pd
import os
import glob

csvs = []
parent_dir = 'demand'
for csv_file in glob.glob(os.path.join(parent_dir, '*.csv')):
    csvs.append(csv_file)

li = []
for filename in csvs:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

df = pd.concat(li, axis=0, ignore_index=True)
os.makedirs('Data', exist_ok=True)  
df.to_csv('Data/demand.csv')  
df.head()

,Time,Day ahead forecast,Hour ahead forecast,Current demand
0,00:00,24159.0,23029.0,22801.0
1,00:05,22844.0,22654.0,22719.0
2,00:10,22844.0,22654.0,22660.0
3,00:15,22844.0,22654.0,22503.0
4,00:20,22844.0,22364.0,22352.0


In [23]:
df = pd.read_csv('Data/demand.csv')
df[300:305]

,Unnamed: 0,Time,Day ahead forecast,Hour ahead forecast,Current demand
300,300,00:55,20740.0,20107.0,20205.0
301,301,01:00,20740.0,20107.0,20134.0
302,302,01:05,20064.0,19948.0,20095.0
303,303,01:10,20064.0,19948.0,20072.0
304,304,01:15,20064.0,19948.0,20008.0
